In [1]:
import topic_model
import os
import pickle
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from gensim.models import ldamodel

%matplotlib inline

### Load the data

In [2]:
unigrams = topic_model.load_data(corpus_file='models/unigrams_corpus.mm', dictionary_file='models/unigrams_dictionary.pkl')
bigrams = topic_model.load_data(corpus_file='models/bigrams_corpus.mm', dictionary_file='models/bigrams_dictionary.pkl')
trigrams = topic_model.load_data(corpus_file='models/trigrams_corpus.mm', dictionary_file='models/trigrams_dictionary.pkl')

No model file input. Skipping model loading.
No model file input. Skipping model loading.
No model file input. Skipping model loading.


### Load it up!

In [3]:
topic_range = np.arange(10, 41, 10)
num_iter = 200
passes = 5
n_words = 20
topic_models = [('unigrams', unigrams[0], unigrams[1]),
                ('bigrams', bigrams[0], bigrams[1]),
                ('trigrams', trigrams[0], trigrams[1])
                ]

print ("Everything is all set to get going!")

Everything is all set to get going!


### Run the models

#### After running each of the n-gram models they are purgred (hopefully) from memory to free up some space and then saved to the 'models/' folder for later coherence analysis
###### The logging output is removed before uploading because it's yuuuge!

In [ ]:
# UNIGRAMS
(topic, topic_corp, topic_dict) = topic_models[0]
print (topic)
models = dict()
files = dict()
for n_topics in topic_range:
    file_path = str(topic) + "_LDA_" + str(n_topics) + "_topics_" + str(num_iter) + "iter_" + str(passes) + "_passes"
    if os.path.isfile(file_path):
        print (n_topics, "topics already run")
    else:
        print (n_topics, "topics running")
        model = topic_model.run_lda(topic_corp, topic_dict, n_topics=n_topics, num_iter=num_iter, rand=10, topic=topic, passes=passes)
        models[topic+' '+str(n_topics)+' topics'] = model
        files[topic+' '+str(n_topics)+' topics'] = file_path
        '''
        print ("Creating the visualization!")
        lda_test_viz.plot_lda(model, n_topics=n_topics, num_iter=num_iter, n_words=n_words, topic=topic)
        pickle.dump(model, open(file_path, "wb"))
        '''
print ("Done")

In [67]:
models_loaded = {}
for i in np.arange(10,101,10):
    fn = 'models/unigrams_LDA_'+str(i)+'_topics_200iter_5_passes'
    with open(fn + '.json', 'r') as model_id2word:
            id2word = json.load(model_id2word)
    model = ldamodel.LdaModel.load(fn+'.gensim',mmap='r')
    model.id2word = id2word
    models_loaded['unigrams '+str(i)+' topics'] = model
models_loaded

{'unigrams 10 topics': <gensim.models.ldamodel.LdaModel at 0x1ab7286908>,
 'unigrams 100 topics': <gensim.models.ldamodel.LdaModel at 0x1d9f30aef0>,
 'unigrams 20 topics': <gensim.models.ldamodel.LdaModel at 0x1abde88ba8>,
 'unigrams 30 topics': <gensim.models.ldamodel.LdaModel at 0x1ac8bb7400>,
 'unigrams 40 topics': <gensim.models.ldamodel.LdaModel at 0x1be58cdc50>,
 'unigrams 50 topics': <gensim.models.ldamodel.LdaModel at 0x1d2df1a4e0>,
 'unigrams 60 topics': <gensim.models.ldamodel.LdaModel at 0x1d5d532d30>,
 'unigrams 70 topics': <gensim.models.ldamodel.LdaModel at 0x1d64f615c0>,
 'unigrams 80 topics': <gensim.models.ldamodel.LdaModel at 0x1d8345fe10>,
 'unigrams 90 topics': <gensim.models.ldamodel.LdaModel at 0x1d8a94a6a0>}

In [32]:
for k in models_loaded.keys():
    print(k, models_loaded[k])

unigrams 10 topics LdaModel(num_terms=677469, num_topics=10, decay=0.5, chunksize=10000)
unigrams 20 topics LdaModel(num_terms=677469, num_topics=20, decay=0.5, chunksize=10000)
unigrams 30 topics LdaModel(num_terms=677469, num_topics=30, decay=0.5, chunksize=10000)
unigrams 40 topics LdaModel(num_terms=677469, num_topics=40, decay=0.5, chunksize=10000)
unigrams 50 topics LdaModel(num_terms=677469, num_topics=50, decay=0.5, chunksize=10000)
unigrams 60 topics LdaModel(num_terms=677469, num_topics=60, decay=0.5, chunksize=10000)
unigrams 70 topics LdaModel(num_terms=677469, num_topics=70, decay=0.5, chunksize=10000)
unigrams 80 topics LdaModel(num_terms=677469, num_topics=80, decay=0.5, chunksize=10000)
unigrams 90 topics LdaModel(num_terms=677469, num_topics=90, decay=0.5, chunksize=10000)
unigrams 100 topics LdaModel(num_terms=677469, num_topics=100, decay=0.5, chunksize=10000)


## Coherence Measuring

Here we will use the UMass, C_V, and Word2Vec coherence metrics to find our optimal solution. In this case it happens to be that 60 topics does well across all three scores indicating that there should be some good topics. However, because it happens to fall between two other topics that perform quite well we will search around 60 topics to ensure we have the best solution.

In [78]:
coherences_umass = {}
coherences_cv = {}
coherences_w2v = {}
for k in models_loaded.keys():
    cm_umass = CoherenceModel(model=models_loaded[k], dictionary=unigrams[1], corpus=unigrams[0], coherence='u_mass')
    cm_cv = CoherenceModel(model=models_loaded[k], texts=texts, dictionary=unigrams[1], corpus=unigrams[0], coherence='c_v')
    cm_w2v = CoherenceModel(model=models_loaded[k], texts=texts, dictionary=unigrams[1], corpus=unigrams[0], coherence='c_w2v')
    
    coherences_umass[k] = cm_umass
    coherences_cv[k] = cm_cv
    coherences_w2v[k] = cm_w2v

In [79]:
for k in coherences_umass.keys():
    print ('%s UMass Coherence: %f C_V Coherece: %f Word2Vec Coherence: %f' %(k, coherences_umass[k].get_coherence(), coherences_cv[k].get_coherence(), coherences_w2v[k].get_coherence()))

unigrams 10 topics UMass Coherence: -16.812155 C_V Coherece: 0.688044 Word2Vec Coherence: 0.800473
unigrams 20 topics UMass Coherence: -16.811304 C_V Coherece: 0.679309 Word2Vec Coherence: 0.806356
unigrams 30 topics UMass Coherence: -16.816029 C_V Coherece: 0.633843 Word2Vec Coherence: 0.798108
unigrams 40 topics UMass Coherence: -16.673742 C_V Coherece: 0.642888 Word2Vec Coherence: 0.800645
unigrams 50 topics UMass Coherence: -16.696102 C_V Coherece: 0.631787 Word2Vec Coherence: 0.795823
unigrams 60 topics UMass Coherence: -16.617950 C_V Coherece: 0.619811 Word2Vec Coherence: 0.796107
unigrams 70 topics UMass Coherence: -16.741522 C_V Coherece: 0.632553 Word2Vec Coherence: 0.798931
unigrams 80 topics UMass Coherence: -16.636924 C_V Coherece: 0.611625 Word2Vec Coherence: 0.796959
unigrams 90 topics UMass Coherence: -16.665079 C_V Coherece: 0.613720 Word2Vec Coherence: 0.793409
unigrams 100 topics UMass Coherence: -16.730660 C_V Coherece: 0.611262 Word2Vec Coherence: 0.799216


In [82]:
ix=0
for umass, cv, w2v in zip(coherences_umass['unigrams 60 topics'].get_coherence_per_topic(), coherences_cv['unigrams 60 topics'].get_coherence_per_topic(), coherences_w2v['unigrams 60 topics'].get_coherence_per_topic()):
    print ("Topic %s UMass Coherence: %f C_V Coherence: %f Word2Vec Coherence: %f" %(str(ix), umass, cv, w2v))
    ix+=1

Topic 0 UMass Coherence: -16.152680 C_V Coherence: 0.631978 Word2Vec Coherence: 0.814092
Topic 1 UMass Coherence: -17.564797 C_V Coherence: 0.587617 Word2Vec Coherence: 0.816171
Topic 2 UMass Coherence: -17.177555 C_V Coherence: 0.651367 Word2Vec Coherence: 0.815071
Topic 3 UMass Coherence: -16.529777 C_V Coherence: 0.635285 Word2Vec Coherence: 0.783009
Topic 4 UMass Coherence: -17.403249 C_V Coherence: 0.628992 Word2Vec Coherence: 0.801418
Topic 5 UMass Coherence: -16.862209 C_V Coherence: 0.628125 Word2Vec Coherence: 0.777190
Topic 6 UMass Coherence: -17.257668 C_V Coherence: 0.536940 Word2Vec Coherence: 0.811749
Topic 7 UMass Coherence: -16.287729 C_V Coherence: 0.649924 Word2Vec Coherence: 0.849626
Topic 8 UMass Coherence: -15.878649 C_V Coherence: 0.613062 Word2Vec Coherence: 0.779287
Topic 9 UMass Coherence: -16.678827 C_V Coherence: 0.655819 Word2Vec Coherence: 0.801923
Topic 10 UMass Coherence: -16.650657 C_V Coherence: 0.548264 Word2Vec Coherence: 0.820738
Topic 11 UMass Coher

#### Ensure 60 is the best running the for n_topics = 58, 60, and 62 and set passes = 10 to get better convergence.

NOTE: the kernel is restarted to clear up the memory.

In [3]:
unigrams = topic_model.load_data(corpus_file='models/unigrams_corpus.mm', dictionary_file='models/unigrams_dictionary.pkl')
texts = pickle.load(open('raw_docs_unigrams.pkl','rb'))

No model file input. Skipping model loading.


In [6]:
topic_range = [58,60,62]
num_iter = 200
passes = 10
n_words = 20
topic_models = ('unigrams', unigrams[0], unigrams[1])
print ("Everything is all set to get going!")

(topic, topic_corp, topic_dict) = topic_models
print (topic)
models = dict()
files = dict()
for n_topics in topic_range:
    file_path = str(topic) + "_LDA_" + str(n_topics) + "_topics_" + str(num_iter) + "iter_" + str(passes) + "_passes"
    if os.path.isfile(file_path):
        print (n_topics, "topics already run")
    else:
        print (n_topics, "topics running")
        model = topic_model.run_lda(topic_corp, topic_dict, n_topics=n_topics, num_iter=num_iter, rand=10, topic=topic, passes=passes, dist=True, workers=3)
        models[topic+' '+str(n_topics)+' topics'] = model
        files[topic+' '+str(n_topics)+' topics'] = file_path
        '''
        print ("Creating the visualization!")
        lda_test_viz.plot_lda(model, n_topics=n_topics, num_iter=num_iter, n_words=n_words, topic=topic)
        pickle.dump(model, open(file_path, "wb"))
        '''
print ("Done")

2017-12-06 22:49:46,898 : INFO : using symmetric alpha at 0.017241379310344827
2017-12-06 22:49:46,998 : INFO : using serial LDA version on this node


Everything is all set to get going!
unigrams
58 topics running


2017-12-06 22:52:28,978 : INFO : running online LDA training, 58 topics, 10 passes over the supplied corpus of 553086 documents, updating every 30000 documents, evaluating every ~300000 documents, iterating 200x with a convergence threshold of 0.001000
2017-12-06 22:52:28,979 : INFO : training LDA model using 3 processes
2017-12-06 22:52:29,200 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/553086, outstanding queue size 1
2017-12-06 22:52:31,249 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/553086, outstanding queue size 2
2017-12-06 22:52:31,434 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/553086, outstanding queue size 3
2017-12-06 22:52:31,622 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #40000/553086, outstanding queue size 4
2017-12-06 22:52:31,812 : INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up to #50000/553086, outstanding queue size 5
2017-12-06 22:52:31,999 : INFO :

2017-12-06 22:55:45,955 : INFO : PROGRESS: pass 0, dispatched chunk #23 = documents up to #240000/553086, outstanding queue size 2
2017-12-06 22:55:46,166 : INFO : PROGRESS: pass 0, dispatched chunk #24 = documents up to #250000/553086, outstanding queue size 3
2017-12-06 22:55:46,374 : INFO : PROGRESS: pass 0, dispatched chunk #25 = documents up to #260000/553086, outstanding queue size 4
2017-12-06 22:55:46,557 : INFO : PROGRESS: pass 0, dispatched chunk #26 = documents up to #270000/553086, outstanding queue size 5
2017-12-06 22:55:46,817 : INFO : PROGRESS: pass 0, dispatched chunk #27 = documents up to #280000/553086, outstanding queue size 6
2017-12-06 22:55:48,642 : INFO : PROGRESS: pass 0, dispatched chunk #28 = documents up to #290000/553086, outstanding queue size 7
2017-12-06 22:55:51,839 : INFO : PROGRESS: pass 0, dispatched chunk #29 = documents up to #300000/553086, outstanding queue size 8
2017-12-06 22:56:06,493 : INFO : merging changes from 30000 documents into a model 

2017-12-06 22:58:58,755 : INFO : PROGRESS: pass 0, dispatched chunk #48 = documents up to #490000/553086, outstanding queue size 7
2017-12-06 22:59:05,325 : INFO : PROGRESS: pass 0, dispatched chunk #49 = documents up to #500000/553086, outstanding queue size 8
2017-12-06 22:59:13,195 : INFO : PROGRESS: pass 0, dispatched chunk #50 = documents up to #510000/553086, outstanding queue size 8
2017-12-06 22:59:19,444 : INFO : PROGRESS: pass 0, dispatched chunk #51 = documents up to #520000/553086, outstanding queue size 8
2017-12-06 22:59:26,833 : INFO : merging changes from 30000 documents into a model of 553086 documents
2017-12-06 22:59:34,470 : INFO : topic #14 (0.017): 0.044*"wed_dress" + 0.017*"!" + 0.014*"?" + 0.012*"gun" + 0.009*"dress" + 0.007*"..." + 0.006*"bless" + 0.006*"new" + 0.006*"post" + 0.005*"album"
2017-12-06 22:59:34,481 : INFO : topic #19 (0.017): 0.022*"!" + 0.017*"?" + 0.009*"..." + 0.008*"gun" + 0.008*"minist" + 0.008*"fuck" + 0.007*"scam" + 0.006*"sea_level" + 0.0

...

2017-12-07 02:57:58,390 : INFO : topic #45 (0.016): 0.038*"marriag" + 0.023*"..." + 0.022*"marri" + 0.014*"summit" + 0.013*"wear" + 0.011*"gorgeous" + 0.010*"challeng" + 0.010*"protest" + 0.009*"hot" + 0.006*"form"
2017-12-07 02:57:58,404 : INFO : topic #59 (0.016): 0.027*"climat_chang" + 0.023*"due" + 0.015*"progress" + 0.013*"#pari" + 0.011*"sander" + 0.008*"bullshit" + 0.007*"straight" + 0.007*"canadian" + 0.006*"per" + 0.006*"#terror"
2017-12-07 02:57:58,416 : INFO : topic #44 (0.016): 0.032*"climat_chang" + 0.021*"#actonclim" + 0.014*"record" + 0.014*"..." + 0.013*"via_@dailycal" + 0.012*"daughter" + 0.012*"connect" + 0.011*"rise" + 0.010*"hillari_clinton" + 0.010*"gun_sale"
2017-12-07 02:57:58,661 : INFO : topic diff=0.113914, rho=0.124700
2017-12-07 02:58:02,200 : INFO : PROGRESS: pass 8, dispatched chunk #36 = documents up to #370000/553086, outstanding queue size 1
2017-12-07 02:58:04,128 : INFO : PROGRESS: pass 8, dispatched chunk #37 = documents up to #380000/553086, outstan

2017-12-07 03:01:18,856 : INFO : topic diff=0.107519, rho=0.124700
2017-12-07 03:01:28,900 : INFO : -9.028 per-word bound, 521.9 perplexity estimate based on a held-out corpus of 3086 documents with 27548 words
2017-12-07 03:01:29,112 : INFO : PROGRESS: pass 9, dispatched chunk #0 = documents up to #10000/553086, outstanding queue size 1
2017-12-07 03:01:31,038 : INFO : PROGRESS: pass 9, dispatched chunk #1 = documents up to #20000/553086, outstanding queue size 2
2017-12-07 03:01:31,219 : INFO : PROGRESS: pass 9, dispatched chunk #2 = documents up to #30000/553086, outstanding queue size 3
2017-12-07 03:01:31,403 : INFO : PROGRESS: pass 9, dispatched chunk #3 = documents up to #40000/553086, outstanding queue size 4
2017-12-07 03:01:31,579 : INFO : PROGRESS: pass 9, dispatched chunk #4 = documents up to #50000/553086, outstanding queue size 5
2017-12-07 03:01:31,760 : INFO : PROGRESS: pass 9, dispatched chunk #5 = documents up to #60000/553086, outstanding queue size 6
2017-12-07 03:0

2017-12-07 03:04:32,994 : INFO : PROGRESS: pass 9, dispatched chunk #22 = documents up to #230000/553086, outstanding queue size 4
2017-12-07 03:04:33,166 : INFO : PROGRESS: pass 9, dispatched chunk #23 = documents up to #240000/553086, outstanding queue size 5
2017-12-07 03:04:33,341 : INFO : PROGRESS: pass 9, dispatched chunk #24 = documents up to #250000/553086, outstanding queue size 6
2017-12-07 03:04:34,890 : INFO : PROGRESS: pass 9, dispatched chunk #25 = documents up to #260000/553086, outstanding queue size 7
2017-12-07 03:04:51,299 : INFO : PROGRESS: pass 9, dispatched chunk #26 = documents up to #270000/553086, outstanding queue size 7
2017-12-07 03:04:51,485 : INFO : PROGRESS: pass 9, dispatched chunk #27 = documents up to #280000/553086, outstanding queue size 8
2017-12-07 03:05:02,879 : INFO : merging changes from 30000 documents into a model of 553086 documents
2017-12-07 03:05:10,477 : INFO : topic #59 (0.016): 0.024*"due" + 0.024*"climat_chang" + 0.015*"progress" + 0.0

2017-12-07 03:07:53,539 : INFO : merging changes from 30000 documents into a model of 553086 documents
2017-12-07 03:08:07,705 : INFO : topic #20 (0.016): 0.059*"action" + 0.029*"climat_chang" + 0.018*"engag" + 0.018*"affect" + 0.013*"#climatechang_start" + 0.013*"rt_music" + 0.013*"video_inspir" + 0.011*"<u+0098>_<u+00ad>" + 0.010*"obama" + 0.010*"50_year"
2017-12-07 03:08:07,718 : INFO : topic #29 (0.016): 0.192*"marriag" + 0.038*"love" + 0.024*"..." + 0.017*"sofia_vergara" + 0.011*"joe_manganiello" + 0.010*"girl" + 0.007*"happi" + 0.006*"marri" + 0.006*"one" + 0.006*"via"
2017-12-07 03:08:07,732 : INFO : topic #43 (0.016): 0.033*"gun" + 0.013*"denial" + 0.012*"line" + 0.012*"@berniesand" + 0.011*"scandal" + 0.010*"intern" + 0.010*"climat_chang" + 0.010*"link_pari" + 0.009*"arm_dealer" + 0.008*"theme"
2017-12-07 03:08:07,745 : INFO : topic #14 (0.016): 0.061*"warm" + 0.045*"coupl" + 0.014*"ice" + 0.013*"..." + 0.012*"natur" + 0.009*"gun" + 0.009*"three" + 0.007*"gone" + 0.007*"theori

LdaModel(num_terms=658846, num_topics=62, decay=0.5, chunksize=10000)
Runtime: 1:27:52.395634
Done


In [9]:
coherences_umass = {}
coherences_cv = {}
coherences_w2v = {}
for k in models.keys():
    cm_umass = CoherenceModel(model=models[k], dictionary=unigrams[1], corpus=unigrams[0], coherence='u_mass')
    cm_cv = CoherenceModel(model=models[k], texts=texts, dictionary=unigrams[1], corpus=unigrams[0], coherence='c_v')
    cm_w2v = CoherenceModel(model=models[k], texts=texts, dictionary=unigrams[1], corpus=unigrams[0], coherence='c_w2v')
    
    coherences_umass[k] = cm_umass
    coherences_cv[k] = cm_cv
    coherences_w2v[k] = cm_w2v

2017-12-07 07:24:18,550 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2017-12-07 07:24:18,573 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2017-12-07 07:24:18,600 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2017-12-07 07:24:18,635 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2017-12-07 07:24:18,661 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2017-12-07 07:24:18,688 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2017-12-07 07:24:18,713 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2017-12-07 07:24:18,746 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2017-12-07 07:24:18,781 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2017-12-07 07:24:18,816 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2017-12-07 07:24:18,852 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2017-12-07 07:24:18

2017-12-07 07:24:21,480 : INFO : CorpusAccumulator accumulated stats from 93000 documents
2017-12-07 07:24:21,514 : INFO : CorpusAccumulator accumulated stats from 94000 documents
2017-12-07 07:24:21,547 : INFO : CorpusAccumulator accumulated stats from 95000 documents
2017-12-07 07:24:21,583 : INFO : CorpusAccumulator accumulated stats from 96000 documents
2017-12-07 07:24:21,618 : INFO : CorpusAccumulator accumulated stats from 97000 documents
2017-12-07 07:24:21,651 : INFO : CorpusAccumulator accumulated stats from 98000 documents
2017-12-07 07:24:21,686 : INFO : CorpusAccumulator accumulated stats from 99000 documents
2017-12-07 07:24:21,722 : INFO : CorpusAccumulator accumulated stats from 100000 documents
2017-12-07 07:24:21,759 : INFO : CorpusAccumulator accumulated stats from 101000 documents
2017-12-07 07:24:21,798 : INFO : CorpusAccumulator accumulated stats from 102000 documents
2017-12-07 07:24:21,832 : INFO : CorpusAccumulator accumulated stats from 103000 documents
2017-1

2017-12-07 07:24:24,369 : INFO : CorpusAccumulator accumulated stats from 184000 documents
2017-12-07 07:24:24,407 : INFO : CorpusAccumulator accumulated stats from 185000 documents
2017-12-07 07:24:24,448 : INFO : CorpusAccumulator accumulated stats from 186000 documents
2017-12-07 07:24:24,485 : INFO : CorpusAccumulator accumulated stats from 187000 documents
2017-12-07 07:24:24,520 : INFO : CorpusAccumulator accumulated stats from 188000 documents
2017-12-07 07:24:24,554 : INFO : CorpusAccumulator accumulated stats from 189000 documents
2017-12-07 07:24:24,576 : INFO : CorpusAccumulator accumulated stats from 190000 documents
2017-12-07 07:24:24,605 : INFO : CorpusAccumulator accumulated stats from 191000 documents
2017-12-07 07:24:24,633 : INFO : CorpusAccumulator accumulated stats from 192000 documents
2017-12-07 07:24:24,657 : INFO : CorpusAccumulator accumulated stats from 193000 documents
2017-12-07 07:24:24,684 : INFO : CorpusAccumulator accumulated stats from 194000 documents

2017-12-07 07:24:27,157 : INFO : CorpusAccumulator accumulated stats from 275000 documents
2017-12-07 07:24:27,191 : INFO : CorpusAccumulator accumulated stats from 276000 documents
2017-12-07 07:24:27,219 : INFO : CorpusAccumulator accumulated stats from 277000 documents
2017-12-07 07:24:27,249 : INFO : CorpusAccumulator accumulated stats from 278000 documents
2017-12-07 07:24:27,279 : INFO : CorpusAccumulator accumulated stats from 279000 documents
2017-12-07 07:24:27,308 : INFO : CorpusAccumulator accumulated stats from 280000 documents
2017-12-07 07:24:27,336 : INFO : CorpusAccumulator accumulated stats from 281000 documents
2017-12-07 07:24:27,364 : INFO : CorpusAccumulator accumulated stats from 282000 documents
2017-12-07 07:24:27,393 : INFO : CorpusAccumulator accumulated stats from 283000 documents
2017-12-07 07:24:27,425 : INFO : CorpusAccumulator accumulated stats from 284000 documents
2017-12-07 07:24:27,451 : INFO : CorpusAccumulator accumulated stats from 285000 documents

2017-12-07 07:24:29,743 : INFO : CorpusAccumulator accumulated stats from 366000 documents
2017-12-07 07:24:29,765 : INFO : CorpusAccumulator accumulated stats from 367000 documents
2017-12-07 07:24:29,796 : INFO : CorpusAccumulator accumulated stats from 368000 documents
2017-12-07 07:24:29,822 : INFO : CorpusAccumulator accumulated stats from 369000 documents
2017-12-07 07:24:29,850 : INFO : CorpusAccumulator accumulated stats from 370000 documents
2017-12-07 07:24:29,879 : INFO : CorpusAccumulator accumulated stats from 371000 documents
2017-12-07 07:24:29,904 : INFO : CorpusAccumulator accumulated stats from 372000 documents
2017-12-07 07:24:29,936 : INFO : CorpusAccumulator accumulated stats from 373000 documents
2017-12-07 07:24:29,967 : INFO : CorpusAccumulator accumulated stats from 374000 documents
2017-12-07 07:24:29,999 : INFO : CorpusAccumulator accumulated stats from 375000 documents
2017-12-07 07:24:30,034 : INFO : CorpusAccumulator accumulated stats from 376000 documents

...

2017-12-07 07:28:03,232 : INFO : CorpusAccumulator accumulated stats from 372000 documents
2017-12-07 07:28:03,255 : INFO : CorpusAccumulator accumulated stats from 373000 documents
2017-12-07 07:28:03,278 : INFO : CorpusAccumulator accumulated stats from 374000 documents
2017-12-07 07:28:03,306 : INFO : CorpusAccumulator accumulated stats from 375000 documents
2017-12-07 07:28:03,333 : INFO : CorpusAccumulator accumulated stats from 376000 documents
2017-12-07 07:28:03,368 : INFO : CorpusAccumulator accumulated stats from 377000 documents
2017-12-07 07:28:03,398 : INFO : CorpusAccumulator accumulated stats from 378000 documents
2017-12-07 07:28:03,421 : INFO : CorpusAccumulator accumulated stats from 379000 documents
2017-12-07 07:28:03,448 : INFO : CorpusAccumulator accumulated stats from 380000 documents
2017-12-07 07:28:03,480 : INFO : CorpusAccumulator accumulated stats from 381000 documents
2017-12-07 07:28:03,507 : INFO : CorpusAccumulator accumulated stats from 382000 documents

2017-12-07 07:28:05,722 : INFO : CorpusAccumulator accumulated stats from 463000 documents
2017-12-07 07:28:05,749 : INFO : CorpusAccumulator accumulated stats from 464000 documents
2017-12-07 07:28:05,781 : INFO : CorpusAccumulator accumulated stats from 465000 documents
2017-12-07 07:28:05,804 : INFO : CorpusAccumulator accumulated stats from 466000 documents
2017-12-07 07:28:05,833 : INFO : CorpusAccumulator accumulated stats from 467000 documents
2017-12-07 07:28:05,865 : INFO : CorpusAccumulator accumulated stats from 468000 documents
2017-12-07 07:28:05,889 : INFO : CorpusAccumulator accumulated stats from 469000 documents
2017-12-07 07:28:05,916 : INFO : CorpusAccumulator accumulated stats from 470000 documents
2017-12-07 07:28:05,943 : INFO : CorpusAccumulator accumulated stats from 471000 documents
2017-12-07 07:28:05,969 : INFO : CorpusAccumulator accumulated stats from 472000 documents
2017-12-07 07:28:05,997 : INFO : CorpusAccumulator accumulated stats from 473000 documents

2017-12-07 07:28:09,008 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows
2017-12-07 07:28:17,748 : INFO : serializing accumulator to return to master...
2017-12-07 07:28:17,750 : INFO : serializing accumulator to return to master...
2017-12-07 07:28:17,750 : INFO : serializing accumulator to return to master...
2017-12-07 07:28:17,751 : INFO : serializing accumulator to return to master...
2017-12-07 07:28:17,752 : INFO : serializing accumulator to return to master...
2017-12-07 07:28:17,751 : INFO : serializing accumulator to return to master...
2017-12-07 07:28:17,753 : INFO : serializing accumulator to return to master...
2017-12-07 07:28:17,763 : INFO : accumulator serialized
2017-12-07 07:28:17,757 : INFO : accumulator serialized
2017-12-07 07:28:17,752 : INFO : accumulator serialized
2017-12-07 07:28:17,754 : INFO : accumulator serialized
2017-12-07 07:28:17,767 : INFO : accumulator serialized
2017-12-07 0

2017-12-07 07:28:32,375 : INFO : sample=0.001 downsamples 10 most-common words
2017-12-07 07:28:32,376 : INFO : downsampling leaves estimated 4156476 word corpus (94.0% of prior 4421199)
2017-12-07 07:28:32,378 : INFO : estimated required memory for 677469 words and 100 dimensions: 880709700 bytes
2017-12-07 07:28:33,973 : INFO : resetting layer weights
2017-12-07 07:28:40,549 : INFO : training model with 7 workers on 677469 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2017-12-07 07:28:41,580 : INFO : PROGRESS: at 1.74% examples, 371899 words/s, in_qsize 14, out_qsize 2
2017-12-07 07:28:42,590 : INFO : PROGRESS: at 3.89% examples, 416051 words/s, in_qsize 13, out_qsize 0
2017-12-07 07:28:43,607 : INFO : PROGRESS: at 6.13% examples, 436288 words/s, in_qsize 13, out_qsize 0
2017-12-07 07:28:44,616 : INFO : PROGRESS: at 8.36% examples, 444398 words/s, in_qsize 13, out_qsize 0
2017-12-07 07:28:45,636 : INFO : PROGRESS: at 10.60% examples, 448307 words/s, in

unigrams 62 topics UMass Coherence: -11.461808 C_V Coherece: nan Word2Vec Coherence: 0.718344


In [10]:
for k in coherences_umass.keys():
    print ('%s UMass Coherence: %f C_V Coherece: %f Word2Vec Coherence: %f' %(k, coherences_umass[k].get_coherence(), coherences_cv[k].get_coherence(), coherences_w2v[k].get_coherence()))

/Volumes/CPO/anaconda3/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:126: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/Volumes/CPO/anaconda3/lib/python3.6/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:200: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))
2017-12-07 07:29:28,928 : WARNING : 3 terms for topic 1 are not in word2vec model vocabulary
2017-12-07 07:29:28,939 : WARNING : 4 terms for topic 6 are not in word2vec model vocabulary
2017-12-07 07:29:28,947 : WARNING : 1 terms for topic 10 are not in word2vec model vocabulary
2017-12-07 07:29:28,950 : WARNING : 2 terms for topic 11 are not in word2vec model vocabulary
2017-12-07 07:29:28,954 : WARNING : 2 terms for topic 13 are not in word2vec model vocabulary
2017-12-07 07:29:28,968 : WARNING : 1 terms for topic 20 are not in word2vec m

unigrams 58 topics UMass Coherence: -10.893985 C_V Coherece: nan Word2Vec Coherence: 0.714840


2017-12-07 07:29:33,347 : WARNING : 2 terms for topic 4 are not in word2vec model vocabulary
2017-12-07 07:29:33,383 : WARNING : 1 terms for topic 24 are not in word2vec model vocabulary
2017-12-07 07:29:33,387 : WARNING : 5 terms for topic 25 are not in word2vec model vocabulary
2017-12-07 07:29:33,395 : WARNING : 4 terms for topic 28 are not in word2vec model vocabulary
2017-12-07 07:29:33,401 : WARNING : 1 terms for topic 29 are not in word2vec model vocabulary
2017-12-07 07:29:33,410 : WARNING : 3 terms for topic 33 are not in word2vec model vocabulary
2017-12-07 07:29:33,413 : WARNING : 2 terms for topic 34 are not in word2vec model vocabulary
2017-12-07 07:29:33,416 : WARNING : 1 terms for topic 35 are not in word2vec model vocabulary
2017-12-07 07:29:33,421 : WARNING : 1 terms for topic 37 are not in word2vec model vocabulary
2017-12-07 07:29:33,431 : WARNING : 2 terms for topic 38 are not in word2vec model vocabulary
2017-12-07 07:29:33,475 : WARNING : 1 terms for topic 56 are 

unigrams 60 topics UMass Coherence: -11.321192 C_V Coherece: nan Word2Vec Coherence: 0.716341


2017-12-07 07:29:37,198 : WARNING : 2 terms for topic 4 are not in word2vec model vocabulary
2017-12-07 07:29:37,204 : WARNING : 1 terms for topic 6 are not in word2vec model vocabulary
2017-12-07 07:29:37,210 : WARNING : 4 terms for topic 9 are not in word2vec model vocabulary
2017-12-07 07:29:37,220 : WARNING : 1 terms for topic 15 are not in word2vec model vocabulary
2017-12-07 07:29:37,230 : WARNING : 2 terms for topic 20 are not in word2vec model vocabulary
2017-12-07 07:29:37,239 : WARNING : 1 terms for topic 24 are not in word2vec model vocabulary
2017-12-07 07:29:37,251 : WARNING : 1 terms for topic 26 are not in word2vec model vocabulary
2017-12-07 07:29:37,260 : WARNING : 1 terms for topic 30 are not in word2vec model vocabulary
2017-12-07 07:29:37,271 : WARNING : 1 terms for topic 32 are not in word2vec model vocabulary
2017-12-07 07:29:37,275 : WARNING : 3 terms for topic 33 are not in word2vec model vocabulary
2017-12-07 07:29:37,293 : WARNING : 2 terms for topic 39 are no

unigrams 62 topics UMass Coherence: -11.461808 C_V Coherece: nan Word2Vec Coherence: 0.718344


In [11]:
models

{'unigrams 58 topics': <gensim.models.ldamulticore.LdaMulticore at 0x1a4064e630>,
 'unigrams 60 topics': <gensim.models.ldamulticore.LdaMulticore at 0x1a4064e710>,
 'unigrams 62 topics': <gensim.models.ldamulticore.LdaMulticore at 0x1a4064ef60>}

In [12]:
for k in files.keys():
    pickle.dump(models[k], open(files[k]+'.pkl', 'wb'))

In [65]:
topic_model.cleanup_models(files,models)

2017-12-07 10:48:38,601 : INFO : saving LdaState object under unigrams_LDA_58_topics_200iter_10_passes.gensim.state, separately None
2017-12-07 10:48:38,602 : INFO : storing np array 'sstats' to unigrams_LDA_58_topics_200iter_10_passes.gensim.state.sstats.npy
2017-12-07 10:48:39,133 : INFO : saved unigrams_LDA_58_topics_200iter_10_passes.gensim.state
2017-12-07 10:48:39,138 : INFO : saving LdaMulticore object under unigrams_LDA_58_topics_200iter_10_passes.gensim, separately ['expElogbeta', 'alpha', 'sstats', 'eta']
2017-12-07 10:48:39,139 : INFO : storing np array 'expElogbeta' to unigrams_LDA_58_topics_200iter_10_passes.gensim.expElogbeta.npy
2017-12-07 10:48:39,712 : INFO : storing np array 'alpha' to unigrams_LDA_58_topics_200iter_10_passes.gensim.alpha.npy
2017-12-07 10:48:39,720 : INFO : storing np array 'eta' to unigrams_LDA_58_topics_200iter_10_passes.gensim.eta.npy
2017-12-07 10:48:39,735 : INFO : not storing attribute id2word
2017-12-07 10:48:39,738 : INFO : not storing attrib

Models saved and zipped!


### Info on the multiple ways to load models into the notebook

In [77]:
from gensim.models import ldamulticore

In [82]:
model = ldamulticore.LdaMulticore.load('models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim', mmap='r')

2017-12-07 11:34:07,947 : INFO : loading LdaMulticore object from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim
2017-12-07 11:34:07,950 : INFO : loading expElogbeta from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.expElogbeta.npy with mmap=r
2017-12-07 11:34:07,968 : INFO : loading alpha from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.alpha.npy with mmap=r
2017-12-07 11:34:07,980 : INFO : loading eta from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.eta.npy with mmap=r
2017-12-07 11:34:07,994 : INFO : setting ignored attribute id2word to None
2017-12-07 11:34:07,995 : INFO : setting ignored attribute dispatcher to None
2017-12-07 11:34:07,996 : INFO : setting ignored attribute state to None
2017-12-07 11:34:07,998 : INFO : loaded models/unigrams_LDA_60_topics_200iter_10_passe

In [83]:
model = ldamodel.LdaModel.load('models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim', mmap='r')

2017-12-07 11:34:32,538 : INFO : loading LdaModel object from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim
2017-12-07 11:34:32,540 : INFO : loading expElogbeta from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.expElogbeta.npy with mmap=r
2017-12-07 11:34:32,557 : INFO : loading alpha from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.alpha.npy with mmap=r
2017-12-07 11:34:32,569 : INFO : loading eta from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.eta.npy with mmap=r
2017-12-07 11:34:32,574 : INFO : setting ignored attribute id2word to None
2017-12-07 11:34:32,576 : INFO : setting ignored attribute dispatcher to None
2017-12-07 11:34:32,578 : INFO : setting ignored attribute state to None
2017-12-07 11:34:32,580 : INFO : loaded models/unigrams_LDA_60_topics_200iter_10_passes 2/

In [88]:
model.id2word = topic_model.load_data(dictionary_file='models/unigrams_dictionary.pkl')[0]
model.id2word

2017-12-07 11:35:40,676 : INFO : loading Dictionary object from models/unigrams_dictionary.pkl


No corpus file input. Skipping corpus loading.


2017-12-07 11:35:41,097 : INFO : loaded models/unigrams_dictionary.pkl


No model file input. Skipping model loading.


In [91]:
model = pickle.load(open('unigrams_LDA_60_topics_200iter_10_passes.pkl', 'rb'))

In [92]:
model.id2word

In [95]:
model = topic_model.load_data(model_file='models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim', mmap='r')[0]

2017-12-07 11:40:55,632 : INFO : loading LdaModel object from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim
2017-12-07 11:40:55,638 : INFO : loading expElogbeta from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.expElogbeta.npy with mmap=r
2017-12-07 11:40:55,657 : INFO : loading alpha from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.alpha.npy with mmap=r
2017-12-07 11:40:55,668 : INFO : loading eta from models/unigrams_LDA_60_topics_200iter_10_passes 2/unigrams_LDA_60_topics_200iter_10_passes.gensim.eta.npy with mmap=r
2017-12-07 11:40:55,683 : INFO : setting ignored attribute id2word to None
2017-12-07 11:40:55,688 : INFO : setting ignored attribute dispatcher to None
2017-12-07 11:40:55,690 : INFO : setting ignored attribute state to None
2017-12-07 11:40:55,692 : INFO : loaded models/unigrams_LDA_60_topics_200iter_10_passes 2/

No corpus file input. Skipping corpus loading.
No dictionary file input. Skipping dictionary loading.


In [98]:
model.id2word = topic_model.load_data(dictionary_file='models/unigrams_dictionary.pkl')[0]
model.id2word

2017-12-07 11:43:48,304 : INFO : loading Dictionary object from models/unigrams_dictionary.pkl


No corpus file input. Skipping corpus loading.


2017-12-07 11:43:48,683 : INFO : loaded models/unigrams_dictionary.pkl


No model file input. Skipping model loading.


### Plot 60 topics!

In [128]:
topic_model.plot_lda(model, n_topics=60, num_iter=200, n_words=20, topic='unigrams')

2017-12-07 12:15:27,930 : INFO : topic #0 (0.017): 0.332*"global_warm" + 0.013*"new_york" + 0.013*"..." + 0.009*"via_@dailycal" + 0.009*"time" + 0.008*"caus" + 0.007*"predict" + 0.005*"hoax" + 0.004*"climat_chang" + 0.004*"theori" + 0.004*"excit" + 0.004*"#isi" + 0.004*"obama" + 0.004*"#tcot" + 0.004*"antarct_ice" + 0.003*"#forecast" + 0.003*"develop" + 0.003*"say" + 0.003*"claim" + 0.003*"crowd"
2017-12-07 12:15:27,946 : INFO : topic #1 (0.017): 0.034*"..." + 0.026*"):" + 0.021*"retweet" + 0.012*"#auspol" + 0.011*"climat_chang" + 0.011*"#lgbt" + 0.011*"weekend" + 0.010*"gun" + 0.009*"heat" + 0.009*"flood" + 0.008*"white_hous" + 0.006*"scientif" + 0.006*"g" + 0.006*"shoot" + 0.005*"cnn" + 0.005*"studi_offer" + 0.005*"region" + 0.005*"daili_show" + 0.005*"around_world" + 0.005*"su"
2017-12-07 12:15:27,962 : INFO : topic #2 (0.017): 0.040*"climat_chang" + 0.021*"marriag_equal" + 0.019*"energi" + 0.019*"refuge" + 0.015*"ice" + 0.014*"adapt" + 0.011*"via_@motherjon" + 0.009*"hillari_clinto

2017-12-07 12:15:28,476 : INFO : topic #21 (0.017): 0.029*"climat_chang" + 0.024*"..." + 0.024*"canada" + 0.023*"law" + 0.017*"book" + 0.016*"challeng" + 0.014*"natur" + 0.013*"tweet" + 0.012*"toda" + 0.010*"#tcot_#uniteblu" + 0.008*"rate" + 0.007*"marriag" + 0.007*"inspir" + 0.007*"sound" + 0.006*"john_kerri" + 0.006*"travel" + 0.006*"new" + 0.005*"inform" + 0.005*"trudeau" + 0.005*"unit_nation"
2017-12-07 12:15:28,501 : INFO : topic #22 (0.017): 0.023*"climat_chang" + 0.017*"#environ" + 0.016*"say" + 0.016*"marriag" + 0.012*"accept" + 0.011*"co2" + 0.011*"happi" + 0.011*"ahead" + 0.010*"summit_pari" + 0.009*"mitig" + 0.008*"judg" + 0.007*"..." + 0.007*"new_studi" + 0.007*"form" + 0.006*"earli" + 0.006*"hour" + 0.006*"pope_franci" + 0.006*"legal_bind" + 0.005*"reach" + 0.005*"short"
2017-12-07 12:15:28,535 : INFO : topic #23 (0.017): 0.172*"climat_chang" + 0.041*"terror" + 0.024*"isi" + 0.024*"obama" + 0.021*"berni_sander" + 0.017*"threat" + 0.016*"say" + 0.016*"blame" + 0.015*"..." +

2017-12-07 12:15:29,128 : INFO : topic #41 (0.017): 0.062*"<u+0098>_<u+0082>" + 0.035*"marri" + 0.018*"marriag" + 0.011*"..." + 0.010*"#weather" + 0.009*"singl" + 0.009*"noaa" + 0.009*"polar_bear" + 0.007*"ain't" + 0.007*"teach" + 0.006*"#renew" + 0.006*"<u+0098>_<u+00b3>" + 0.006*"suit" + 0.005*"<u+0098>_<u+00b1>" + 0.005*"#coal" + 0.004*"#china" + 0.004*"spous" + 0.004*"cheat" + 0.004*"<u+0098>_<u+0092>" + 0.004*"pressur"
2017-12-07 12:15:29,202 : INFO : topic #42 (0.017): 0.028*"god" + 0.023*"climat_chang" + 0.016*"..." + 0.012*"marriag" + 0.010*"design" + 0.010*"expert" + 0.010*"fake" + 0.009*"gun" + 0.009*"except" + 0.008*"univers" + 0.008*"dream" + 0.007*"ridicul" + 0.006*"gone" + 0.006*"negoti" + 0.006*"whether" + 0.006*"nov" + 0.006*"gain" + 0.005*"us" + 0.005*"y'all" + 0.005*"90"
2017-12-07 12:15:29,243 : INFO : topic #43 (0.017): 0.044*"gun" + 0.015*"allow" + 0.012*"texa" + 0.009*"peac" + 0.009*"drop" + 0.008*"scandal" + 0.007*"climat_chang" + 0.007*"arm_dealer" + 0.007*"good